In [ ]:
#imports
import warnings
from os import listdir

import random

import cv2
from keras import optimizers
from keras.applications import VGG16, VGG19, InceptionResNetV2
from keras.preprocessing import image as KImage
from keras.applications import imagenet_utils
from keras.applications.vgg19 import preprocess_input
from keras.utils.data_utils import get_file


from PIL import ImageFile, Image

from sklearn.model_selection import train_test_split
import glob

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.backend import sigmoid
from keras.callbacks import EarlyStopping

from keras.callbacks import ModelCheckpoint, Callback
from keras.layers import Activation, BatchNormalization
from keras.layers import Dense
from keras.layers import Dropout, Flatten
from keras.models import Sequential
from keras.utils.generic_utils import get_custom_objects


from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn import metrics
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay, \
    confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder


In [ ]:
#connect google drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!ls /content/gdrive/'My Drive'/'Climate Change BU'/climate_change_dataset/images

In [ ]:
# Swish activation fucntion
def swish(x, beta=1):
    return (x * sigmoid(beta * x))

In [ ]:
# Default all categories
def one_hot_encode(Y):
  #one hot encode
  print(Y.shape)
  print(Y[:10])
  onehotY = np.zeros((Y.shape[0], 20))
  for i in range(0, Y.shape[0]):
    #   if Y[i] == 99:
    #       onehotY[i][20] = 1
    #   else:
        onehotY[i][Y[i] - 1] = 1
  return onehotY

# # reducing the categories to hopefully gain accuracy for now
# land = [1, 2, 3, 13, 14, 17] #  0 landscapes, disater, agriculture, transportation, plants
# animals =  [4]       # 1 animals
# people = [5, 6, 7, 8, 9, 10, 11, 12] # 2 people
# activites = [15, 16] # 3 activites
# inst_ref = [18]  # 4 Institutional Reference
# energy = [19, 20] # 5 Conventional Energy, Green Tech
# # # 99 gets dropped

# def one_hot_encode(Y):
#   #one hot encode
#   onehotY = np.zeros((Y.shape[0], 6))
#   for i in range(0, Y.shape[0]):
#       if Y[i] in land:
#           onehotY[i][0] = 1
#       elif Y[i] in animals:
#           onehotY[i][1] = 1
#       elif Y[i] in people:
#           onehotY[i][2] = 1
#       elif Y[i] in activites:
#           onehotY[i][3] = 1
#       elif Y[i] in inst_ref:
#           onehotY[i][4] = 1
#       elif Y[i] in energy:
#           onehotY[i][5] = 1
#   return onehotY

In [ ]:
# https://medium.com/analytics-vidhya/how-to-apply-data-augmentation-to-deal-with-unbalanced-datasets-in-20-lines-of-code-ada8521320c9
from keras.utils.data_utils import Sequence
from imblearn.over_sampling import RandomOverSampler
from imblearn.keras import balanced_batch_generator

class BalancedDataGenerator(Sequence):
    """ImageDataGenerator + RandomOversampling"""
    def __init__(self, x, y, datagen, batch_size=32):
        self.datagen = datagen
        self.batch_size = min(batch_size, x.shape[0])
        datagen.fit(x)
        self.gen, self.steps_per_epoch = balanced_batch_generator(x.reshape(x.shape[0], -1), y, sampler=RandomOverSampler(), batch_size=self.batch_size, keep_sparse=True)
        self._shape = (self.steps_per_epoch * batch_size, *x.shape[1:])
        
    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, idx):
        x_batch, y_batch = self.gen.__next__()
        x_batch = x_batch.reshape(-1, *self._shape[1:])
        return self.datagen.flow(x_batch, y_batch, batch_size=self.batch_size).next()
   




/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from s

In [ ]:
def translate(img, shift=10, direction='right', roll=True):
    assert direction in ['right', 'left', 'down', 'up'], 'Directions should be top|up|left|right'
    img = img.copy()
    if direction == 'right':
        right_slice = img[:, -shift:].copy()
        img[:, shift:] = img[:, :-shift]
        if roll:
            img[:,:shift] = np.fliplr(right_slice)
    if direction == 'left':
        left_slice = img[:, :shift].copy()
        img[:, :-shift] = img[:, shift:]
        if roll:
            img[:, -shift:] = left_slice
    if direction == 'down':
        down_slice = img[-shift:, :].copy()
        img[shift:, :] = img[:-shift,:]
        if roll:
            img[:shift, :] = down_slice
    if direction == 'up':
        upper_slice = img[:shift, :].copy()
        img[:-shift, :] = img[shift:, :]
        if roll:
            img[-shift:,:] = upper_slice
    return img

def rotate_img(img, angle, bg_patch=(5,5)):
    assert len(img.shape) <= 3, "Incorrect image shape"
    rgb = len(img.shape) == 3
    if rgb:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1], :], axis=(0,1))
    else:
        bg_color = np.mean(img[:bg_patch[0], :bg_patch[1]])
    img = rotate(img, angle, reshape=False)
    mask = [img <= 0, np.any(img <= 0, axis=-1)][rgb]
    img[mask] = bg_color
    return img

def gaussian_noise(img, mean=0, sigma=0.03):
    img = img.copy()
    noise = np.random.normal(mean, sigma, img.shape)
    mask_overflow_upper = img+noise >= 1.0
    mask_overflow_lower = img+noise < 0
    noise[mask_overflow_upper] = 1.0
    noise[mask_overflow_lower] = 0
    img += intnoise
    return img

def augment_data(X, Y):
    # get the max data in a category
    maxItems = (np.bincount(Y)[np.argmax(np.bincount(Y))]) * 2
    #split into sub arrays
    classes = np.zeros((20 , maxItems, 224, 224, 3), dtype=np.uint8)
    values = np.zeros((20 * maxItems), dtype=np.uint8)
    for i in range(1, 21):
        classArray = X[np.where(Y == i)]
        class_len = classArray.shape[0]
        modifiedArray = np.resize(classArray, (maxItems, 224, 224, 3))
        image_index = 0
        for j in range(class_len, maxItems):
            if image_index > class_len:
                image_index = 0
            if image_index % 10 == 0:
                img = classArray[image_index % class_len]
                img = translate(img, shift = random.randint(10,25))
                # img = gaussian_noise(img)
            if image_index % 10 == 1:
                img = classArray[image_index % class_len]
                img = translate(img, 'left', shift = random.randint(10,25))
                # img = gaussian_noise(img)
            if image_index % 10 == 2:
                img = classArray[image_index % class_len]
                img = translate(img, 'up', shift = random.randint(10,25))
                # img = gaussian_noise(img)
            if image_index % 10 == 3:
                img = classArray[image_index % class_len]
                img = translate(img,'down', shift = random.randint(10,25))
                # img = gaussian_noise(img)
            if image_index % 10 == 4:
                img = classArray[image_index % class_len]
                img = rotate(img, angle = random.randint(5,25))
                # img = gaussian_noise(img)
            if image_index % 10 == 5:
                img = classArray[image_index % class_len]
                img = rotate(img, angle = random.randint(5,25))
                # img = gaussian_noise(img)
            if image_index % 10 == 6:
                img = classArray[image_index % class_len]
                img = rotate(img, angle = random.randint(5,25))
                # img = gaussian_noise(img)
            if image_index % 10 == 7:
                img = classArray[image_index % class_len]
                img = rotate(img, angle = random.randint(-25,-5))
                # img = gaussian_noise(img)
            if image_index % 10 == 8:
                img = classArray[image_index % class_len]
                img = rotate(img, angle = random.randint(-25,-5))
                # img = gaussian_noise(img)
            if image_index % 10 == 9:
                img = classArray[image_index % class_len]
                img = rotate(img, angle = random.randint(-25,-5))
                # img = gaussian_noise(img)
            
            modifiedArray[j] = img
        classes[i-1] = modifiedArray
        values[(i-1) * maxItems: i * maxItems] = i

    classes = np.reshape(classes, (20 * maxItems,  224, 224, 3))
    return classes, values

    
    # print(class1)


In [ ]:

# load all images into memory based upon Carly Reardon's code
def load_dataset(path):
    images = []

    # Get label info
    full_label_info = np.genfromtxt("/content/gdrive/My Drive/Climate Change BU/climate_change_dataset/labels.csv", delimiter=',', dtype=np.uint8)
    #grab question 4 labels
    q4_labels = full_label_info[:, 5]
    photos = glob.glob(path+"*.jpg")
    
    # get images in image di
    imagesList = listdir(path)
#TODO split into train test before data augmentation
    deleteRows = []
    for i in range(0, len(imagesList)):
        if q4_labels[i] == 0:
            deleteRows.append(i)
            continue
        if q4_labels[i] == 255:
            deleteRows.append(i)
            continue
        if q4_labels[i] == 99:
            deleteRows.append(i)
            continue
        img = KImage.load_img(path + "/" + imagesList[i], target_size=(224, 224))
        img = KImage.img_to_array(img)
        # img = preprocess_input(img)
        images.append(img)
    X = np.asarray(images, dtype='uint8')

    # remove unreleated images
    Y = np.delete(q4_labels, deleteRows)
    Y = np.delete(Y, -1)


    trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.5, random_state=1)

    # trainX, trainY = augment_data(trainX, trainY)

    for image in trainX:
        image = preprocess_input(image)
    for image in testX:
        image = preprocess_input(image)

    from sklearn.utils import shuffle

    trainX, trainY = shuffle(trainX, trainY, random_state=42)
    print(trainY[:15])

    # one hot encode
    oneHotY = one_hot_encode(trainY)
    trainY = np.asarray(oneHotY, dtype='uint8')

    oneHotY = one_hot_encode(testY)
    testY = np.asarray(oneHotY, dtype='uint8')

    return trainX, trainY, testX, testY

# # Here for testing
# path = "/content/gdrive/My Drive/Climate Change BU/climate_change_dataset/images/"
# trainX, trainY, testX, testY = load_dataset(path)

In [ ]:

#create a model
# Base Line model
def baseline_model(in_shape=(224, 224, 3),  out_shape=21):
    #setup optimizer
    opt = optimizers.Adam(learning_rate=0.0001)

    vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=in_shape)
    # vgg_base = VGG19(include_top=True, input_shape=in_shape)
    vgg_base.trainable = False
    model = Sequential()
    model.add(vgg_base)
    model.add(Flatten())
    # #Layers based upon AlexNet
    # # 1st Fully Connected Layer
    # model.add(Dense(1024))
    # model.add(Activation('relu'))
    # # Add Dropout to prevent overfitting
    # model.add(Dropout(0.4))

    # # 2nd Fully Connected Layer
    # model.add(Dense(512))
    # model.add(Activation('relu'))
    # # Add Dropout
    # model.add(Dropout(0.4))

    # 3rd Fully Connected Layer
    model.add(Dense(256))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.2))

    # Output Layer
    model.add(Dense(out_shape))
    model.add(Activation('softmax'))

    model.summary()

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer="adam",
                          metrics=['accuracy', 'categorical_accuracy'])
    return model

In [ ]:
#places model
#create a model
# Base Line model
# Weights from 
# @misc{gkallia2017keras_places365,
# title={Keras-VGG16-Places365},
# author={Grigorios Kalliatakis},
# year={2017},
# publisher={GitHub},
# howpublished={\url{https://github.com/GKalliatakis/Keras-VGG16-places365}},
# }
def define_places_model(in_shape=(224, 224, 3),  out_shape = 20):
    # enable swish
    get_custom_objects().update({'swish': Activation(swish)})
    
    WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-VGG16-places365/releases/download/v1.0/vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5'
    weights_path = get_file('vgg16-hybrid1365_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
    #setup optimizer
    opt = optimizers.Adam(learning_rate=0.0001)

    vgg_base = VGG16(weights=weights_path, include_top=False, input_shape=in_shape)
    # vgg_base = VGG19(include_top=True, input_shape=in_shape)
    vgg_base.trainable = False
    model = Sequential()
    model.add(vgg_base)
    model.add(Flatten())
    # #Layers based upon AlexNet
    # # 1st Fully Connected Layer
    # model.add(Dense(1024))
    # model.add(Activation('relu'))
    # # Add Dropout to prevent overfitting
    # model.add(Dropout(0.4))

    # 2nd Fully Connected Layer
    model.add(Dense(512))
    model.add(Activation(swish))
    # Add Dropout
    model.add(Dropout(0.4))

    # 3rd Fully Connected Layer
    model.add(Dense(256))
    model.add(Activation(swish))
    # Add Dropout
    model.add(Dropout(0.2))

    # Output Layer
   
    model.add(Dense(out_shape))
    model.add(Activation('softmax'))

    model.summary()

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer="adam",
                          metrics=['accuracy'])
    return model

In [ ]:
# Get data
path = "/content/gdrive/My Drive/Climate Change BU/climate_change_dataset/images/"
trainX, trainY, testX, testY = load_dataset(path)

[ 1 12  6  6  6 13 13  6 13 12  3  2  6  6  6]
(450,)
[ 1 12  6  6  6 13 13  6 13 12]
(450,)
[ 6 19  5  7  5  7 16  6 16  4]


In [ ]:
# Setup up pipeline
epochs = 100
# callbacks = [
#     EarlyStopping(monitor='loss', mode='min',min_delta=.001, verbose=1, patience=8),
#     EarlyStopping(monitor='accuracy', mode='max', min_delta=.0001 ,patience=8)
#     ]
# estimators = []
# estimators.append(('standardize', StandardScaler()))
# estimators.append(('mlp', KerasClassifier(build_fn=places_model, epochs=epochs,
#                                           batch_size=32, verbose=1, callbacks=callbacks)))
# pipeline = Pipeline(estimators)

model = define_places_model()

58916864/58909656 [==============================] - 5s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________

In [ ]:
datagen = ImageDataGenerator()

balanced_gen = BalancedDataGenerator(trainX, trainY, datagen, batch_size=32)

# fit
# model.fit(trainX, trainY, epochs=epochs, batch_size=32, verbose=1)
model.fit_generator(balanced_gen,
          steps_per_epoch=trainX.shape[0] / 32, epochs=epochs, verbose=1)


Instructions for updating:
Please use Model.fit, which supports generators.


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/100
15/14 [================================] - 1s 64ms/step - loss: 18.4220 - accuracy: 0.3021
Epoch 2/100
15/14 [================================] - 1s 64ms/step - loss: 8.9873 - accuracy: 0.5958
Epoch 3/100
15/14 [================================] - 1s 64ms/step - loss: 5.1318 - accuracy: 0.6958
Epoch 4/100
15/14 [================================] - 1s 64ms/step - loss: 5.1186 - accuracy: 0.7167
Epoch 5/100
15/14 [================================] - 1s 64ms/step - loss: 3.9824 - accuracy: 0.7500
Epoch 6/100
15/14 [================================] - 2s 100ms/step - loss: 3.3189 - accuracy: 0.7860
Epoch 7/100
15/14 [================================] - 1s 76ms/step - loss: 3.6964 - accuracy: 0.8083
Epoch 8/100
15/14 [================================] - 1s 63ms/step - loss: 3.3922 - accuracy: 0.7896
Epoch 9/100
15/14 [================================] - 1s 65ms/step - loss: 3.4151 - accuracy: 0.8292
Epoch 10/100
15/14 [================================] - 1s 65ms/step - loss: 2.0

In [ ]:
#print results

results = model.evaluate(testX, testY, batch_size=32, verbose=1)
print("Test Loss, Accuracy: "+str(results))
Ypred = model.predict(testX)
# trainPred = model.predict(trainX)
labels['Natural landscape', 
        'Urban landscape',
        'Disaster',
        'Animal' ,
        'People: Demonstrations',
        'People: Politicians',
        'People—NGO/Activists',
        'People—business people',
        'People—Scientists and researchers',
        'People—Entertainment or sports celebrity',
        'People—Police/other law enforcement officers',
        'People—ordinary citizens ',
        'Agriculture',
        'Transportation',
        'Political activities',
        'Research activities' ,
        'Plants',
        'Institutional Reference',
        'Conventional Energy',
        'Green Tech']
print(classification_report(np.argmax(testY, axis=1), np.argmax(Ypred, axis=1), labels = lables))

15/15 [==============================] - 1s 69ms/step - loss: 25.7131 - accuracy: 0.2089
Test Loss, Accuracy: [25.713077545166016, 0.20888888835906982]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        25
           1       0.00      0.00      0.00         8
           2       0.12      0.07      0.09        28
           3       0.05      0.07      0.06        14
           4       0.26      0.15      0.19        88
           5       0.25      0.64      0.36       113
           6       0.16      0.09      0.12        44
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00        37
          10       0.00      0.00      0.00         5
          11       0.09      0.10      0.10        20
          12       0.00      0.00      0.00         8
          13       0.00      0.00      0.00         9
          14       0.00      0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from mlxtend.evaluate import confusion_matrix
cm = confusion_matrix(y_target=np.argmax(testY, axis=1), 
                      y_predicted=np.argmax(Ypred, axis=1), 
                      binary=False)